### Connection database

In [1]:
import mysql.connector
import random
from faker import Faker
from snowflake import SnowflakeGenerator, Snowflake
from datetime import datetime
import bcrypt

db_config = {
    "host": "localhost",
    "user": "root",
    "password": "123456",
    "database": "optima-project-retail-manager"
}

user_id = 1450492617670998016

fake = Faker('vi_VN')
sf = Snowflake.parse(856165981072306198, 1288834974657)
gen = SnowflakeGenerator.from_snowflake(sf)

### Location Work

### Shift



In [ ]:
shifts_data = [
    {'name': 'Sáng', 'start_time': '07:00:00', 'end_time': '12:00:00'},
    {'name': 'Chiều', 'start_time': '12:00:00', 'end_time': '17:00:00'},
    {'name': 'Tối', 'start_time': '17:00:00', 'end_time': '22:00:00'}
]

def generate_shifts():
    """
    Kết nối đến DB, sinh dữ liệu cho các ca làm việc và chèn vào bảng `shift`.
    """
    try:
        # Kết nối đến MySQL
        connect_mysql = mysql.connector.connect(**db_config)
        cursor = connect_mysql.cursor()

        print("🚀 Bắt đầu sinh dữ liệu cho bảng `shift`...")

        # Lấy thời gian hiện tại
        now = datetime.now()

        # Vòng lặp qua từng ca làm việc để tạo dữ liệu
        for shift in shifts_data:
            # Sinh ngẫu nhiên lương cho mỗi ca (ví dụ từ 150,000 đến 400,000)
            wage = 0
            id = next(gen)
            if shift['name'] == "Sáng":
                wage = 200000
            elif shift['name'] == "Tối":
                wage = 220000
            elif shift['name'] == "Chiều":
                wage = 200000

            # Dữ liệu để chèn vào bảng
            shift_record = (
                id,
                shift['name'],
                shift['start_time'],
                shift['end_time'],
                wage,
                now,         # created_at
                user_id,     # created_by
                now,         # updated_at
                user_id      # updated_by
            )


            sql_insert_query = """
            INSERT INTO shift
            (id, name, start_time, end_time, wage_per_shift, created_at, created_by, updated_at, updated_by)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
            """

            # Thực thi câu lệnh
            cursor.execute(sql_insert_query, shift_record)
            print(f"  -> Đã thêm ca: {shift['name']}")

        # Commit các thay đổi vào database
        connect_mysql.commit()
        print(f"\n✅ Hoàn thành! Đã chèn thành công {cursor.rowcount} ca làm việc vào database.")

    except mysql.connector.Error as error:
        print(f"❌ Lỗi khi kết nối hoặc chèn dữ liệu: {error}")

    finally:
        # Đóng kết nối
        if 'connect_mysql' in locals() and connect_mysql.is_connected():
            cursor.close()
            connect_mysql.close()
            print("📦 Đã đóng kết nối database.")

generate_shifts();

### Role

In [ ]:
# ID của người dùng thực hiện thao tác này
user_id = 1450492617670998016


# --- Dữ liệu vai trò cần thêm ---
roles_to_add = [
    {
        'name': 'ADMIN',
        'title': 'Quản trị viên hệ thống',
        'description': 'Có toàn quyền truy cập và cấu hình hệ thống.'
    },
    {
        'name': 'MANAGER',
        'title': 'Quản lý',
        'description': 'Có quyền quản lý nhân viên, xem báo cáo, cấu hình cửa hàng/kho.'
    },
    {
        'name': 'EMPLOYEE',
        'title': 'Nhân viên',
        'description': 'Có các quyền hạn cơ bản như bán hàng, chấm công, quản lý công việc cá nhân.'
    }
]

def generate_roles():
    """
    Kết nối DB, sinh ID và chèn dữ liệu cho các vai trò vào bảng `role`.
    
    Lưu ý: Schema gốc bị thiếu dấu phẩy, schema đúng phải là:
    CREATE TABLE role (
        id BIGINT PRIMARY KEY,
        title VARCHAR(250),
        name VARCHAR(50) NOT NULL UNIQUE,
        description TEXT,
        created_at DATETIME,
        created_by BIGINT,
        updated_at DATETIME,
        updated_by BIGINT
    );
    """
    try:
        connect_mysql = mysql.connector.connect(**db_config)
        cursor = connect_mysql.cursor()

        print("🚀 Bắt đầu sinh dữ liệu cho bảng `role`...")
        now = datetime.now()

        for role_data in roles_to_add:
            # Tạo một ID mới cho mỗi vai trò
            new_id = next(gen)

            # Dữ liệu để chèn vào bảng
            role_record = (
                new_id,
                role_data['title'],
                role_data['name'],
                role_data['description'],
                now,
                user_id,
                now,
                user_id
            )

            # Câu lệnh SQL INSERT
            sql_insert_query = """
            INSERT INTO role 
            (id, title, name, description, created_at, created_by, updated_at, updated_by)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            """

            cursor.execute(sql_insert_query, role_record)
            print(f"  -> Đã thêm vai trò: '{role_data['name']}' với ID: {new_id}")

        connect_mysql.commit()
        print(f"\n✅ Hoàn thành! Đã chèn thành công {cursor.rowcount} vai trò vào database.")

    except mysql.connector.Error as error:
        print(f"❌ Lỗi khi kết nối hoặc chèn dữ liệu: {error}")

    finally:
        if 'connect_mysql' in locals() and connect_mysql.is_connected():
            cursor.close()
            connect_mysql.close()
            print("📦 Đã đóng kết nối database.")

generate_roles()

### Department (Chưa cần thiết lắm)

### user

In [2]:

# --- Các hàm phụ (giữ nguyên) ---
connect_mysql = mysql.connector.connect(**db_config)
cursor = connect_mysql.cursor()

print("B1")

def get_existing_ids(cursor, table_name):
    """Hàm phụ để lấy danh sách ID từ một bảng."""
    try:
        cursor.execute(f"SELECT id FROM {table_name}")
        print("Buoc2 ")
        return [item[0] for item in cursor.fetchall()]
        
    except mysql.connector.Error as e:
        print(f"Lỗi khi lấy ID từ bảng '{table_name}': {e}")
        return []

def get_roles_map(cursor):
    """Lấy danh sách vai trò và chuyển thành dạng dictionary để dễ tra cứu."""
    try:
        cursor.execute("SELECT id, name FROM role")
        print("buoc3")
        return {name: role_id for role_id, name in cursor.fetchall()}
    except mysql.connector.Error as e:
        print(f"Lỗi khi lấy dữ liệu từ bảng 'role': {e}")
        return {}


# --- Hàm chính ---
def generate_users_per_location():
    manager_count = 1
    employee_count = 1
    
    try:
        print("🚀 Bắt đầu sinh dữ liệu người dùng với logic lương mới...")
        
        # 1. Lấy dữ liệu nền tảng
        print("   -> Lấy ID từ các bảng liên quan...")
        store_ids = get_existing_ids(cursor, 'store')
        warehouse_ids = get_existing_ids(cursor, 'warehouse')
        all_location_ids = store_ids + warehouse_ids
        department_ids = 1
        roles_map = get_roles_map(cursor)
        print("Buoc 4")
        # 2. Kiểm tra dữ liệu nền tảng
        if not all([all_location_ids, department_ids, roles_map]):
            print("❌ Lỗi: Cần có dữ liệu ở các bảng store, warehouse, department và role trước khi chạy.")
            return
            
        manager_role_id = roles_map.get('MANAGER')
        employee_role_id = roles_map.get('EMPLOYEE')

        if not manager_role_id or not employee_role_id:
            print("❌ Lỗi: Không tìm thấy vai trò 'MANAGER' hoặc 'EMPLOYEE' trong bảng role.")
            return

        # 3. Vòng lặp chính qua từng địa điểm
        print(f"   -> Tìm thấy {len(all_location_ids)} địa điểm. Bắt đầu tạo user...")
        hashed_password = bcrypt.hashpw(b"123456", bcrypt.gensalt())
        
        sql_insert_user = """
        INSERT INTO user (id, name, username, password, email, gender, date_of_birth, address, 
                          office_id, department_id, role_id, join_date, salary_type, 
                          salary_per_day, status, created_at, created_by, updated_at, 
                          updated_by, phone)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """

        for location_id in all_location_ids:
            print(f"\n--- Đang xử lý cho địa điểm ID: {location_id} ---")

            # --- TẠO MANAGER ---
            manager_username = f"manager{manager_count}"
            new_user_id = next(gen)
            
            # THAY ĐỔI: Manager luôn có salary_type là 'daily' và lương là số nguyên
            manager_salary_type = 'daily'
            manager_salary = (random.randint(800000, 1000000))
            
            manager_record = (
                new_user_id, fake.name(), manager_username, hashed_password.decode('utf-8'),
                fake.email(), random.choice(['Nam', 'Nữ']), fake.date_of_birth(minimum_age=25, maximum_age=55),
                fake.address(), location_id, department_ids, manager_role_id,
                fake.date_between(start_date='-10y'), manager_salary_type, manager_salary,
                'active', datetime.now(), user_id, datetime.now(), user_id, fake.phone_number()
            )
            cursor.execute(sql_insert_user, manager_record)
            print(f"   ✓ Đã tạo Manager: {manager_username}")
            manager_count += 1
            
            # --- TẠO EMPLOYEE ---
            num_employees = random.randint(10, 20)
            print(f"   -> Sẽ tạo {num_employees} nhân viên cho địa điểm này...")
            for _ in range(num_employees):
                employee_username = f"employee{employee_count}"
                new_user_id = next(gen)
                
                # THAY ĐỔI: Employee có salary_type ngẫu nhiên và xử lý lương tương ứng
                employee_salary_type = random.choice(['daily', 'shift'])
                if employee_salary_type == 'daily':
                    # Lương daily là số nguyên từ 400k - 600k
                    employee_salary = random.randint(400000, 600000)
                else:
                    # Lương shift là 0
                    employee_salary = 0
                
                employee_record = (
                    new_user_id, fake.name(), employee_username, hashed_password.decode('utf-8'),
                    fake.email(), random.choice(['Nam', 'Nữ']), fake.date_of_birth(minimum_age=18, maximum_age=50),
                    fake.address(), location_id, department_ids, employee_role_id,
                    fake.date_between(start_date='-5y'), employee_salary_type, employee_salary,
                    'active', datetime.now(), user_id, datetime.now(), user_id, fake.phone_number()
                )
                
                cursor.execute(sql_insert_user, employee_record)
                employee_count += 1
            print(f"   ✓ Đã tạo xong {num_employees} nhân viên.")

        connect_mysql.commit()
        print(f"\n✅ Hoàn thành! Đã sinh dữ liệu người dùng cho tất cả các địa điểm.")

    except mysql.connector.Error as error:
        print(f"❌ Lỗi khi thực thi: {error}")
    finally:
        if 'connect_mysql' in locals() and connect_mysql.is_connected():
            cursor.close()
            connect_mysql.close()
            print("📦 Đã đóng kết nối database.")
            
generate_users_per_location()

B1
🚀 Bắt đầu sinh dữ liệu người dùng với logic lương mới...
   -> Lấy ID từ các bảng liên quan...
Buoc2 
Buoc2 
buoc3
Buoc 4
   -> Tìm thấy 3903 địa điểm. Bắt đầu tạo user...

--- Đang xử lý cho địa điểm ID: 1971069766866481152 ---
   ✓ Đã tạo Manager: manager1
   -> Sẽ tạo 11 nhân viên cho địa điểm này...
   ✓ Đã tạo xong 11 nhân viên.

--- Đang xử lý cho địa điểm ID: 1971069766866481153 ---
   ✓ Đã tạo Manager: manager2
   -> Sẽ tạo 16 nhân viên cho địa điểm này...
   ✓ Đã tạo xong 16 nhân viên.

--- Đang xử lý cho địa điểm ID: 1971069766866481154 ---
   ✓ Đã tạo Manager: manager3
   -> Sẽ tạo 14 nhân viên cho địa điểm này...
   ✓ Đã tạo xong 14 nhân viên.

--- Đang xử lý cho địa điểm ID: 1971069766866481155 ---
   ✓ Đã tạo Manager: manager4
   -> Sẽ tạo 15 nhân viên cho địa điểm này...
   ✓ Đã tạo xong 15 nhân viên.

--- Đang xử lý cho địa điểm ID: 1971069766866481156 ---
   ✓ Đã tạo Manager: manager5
   -> Sẽ tạo 13 nhân viên cho địa điểm này...
   ✓ Đã tạo xong 13 nhân viên.

--- 

### Thêm admin

In [ ]:

# --- Các hàm phụ (giữ nguyên) ---
connect_mysql = mysql.connector.connect(**db_config)
cursor = connect_mysql.cursor()

print("B1")

def get_existing_ids(cursor, table_name):
    """Hàm phụ để lấy danh sách ID từ một bảng."""
    try:
        cursor.execute(f"SELECT id FROM {table_name}")
        print("Buoc2 ")
        return [item[0] for item in cursor.fetchall()]
        
    except mysql.connector.Error as e:
        print(f"Lỗi khi lấy ID từ bảng '{table_name}': {e}")
        return []

def get_roles_map(cursor):
    """Lấy danh sách vai trò và chuyển thành dạng dictionary để dễ tra cứu."""
    try:
        cursor.execute("SELECT id, name FROM role")
        print("buoc3")
        return {name: role_id for role_id, name in cursor.fetchall()}
    except mysql.connector.Error as e:
        print(f"Lỗi khi lấy dữ liệu từ bảng 'role': {e}")
        return {}


def generate_admin_users():
    admin_count = 1
    try:
        connect_mysql = mysql.connector.connect(**db_config)
        cursor = connect_mysql.cursor()
        print("🚀 Bắt đầu sinh dữ liệu cho người dùng ADMIN...")
        
        # 1. Lấy dữ liệu nền tảng
        roles_map = get_roles_map(cursor)
        admin_role_id = roles_map.get('ADMIN')

        # 2. Chuẩn bị câu lệnh SQL và mật khẩu
        hashed_password = bcrypt.hashpw(b"123456", bcrypt.gensalt())
        
        # **SỬA LỖI CHÍNH**: Thêm cột `salary_per_day` vào câu lệnh INSERT (đủ 20 cột)
        sql_insert_user = """
        INSERT INTO user (id, name, username, password, email, gender, date_of_birth, address, 
                        office_id, department_id, role_id, join_date, salary_type, 
                        salary_per_day, status, created_at, created_by, updated_at, 
                        updated_by, phone)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """

        # 3. Vòng lặp để tạo 9 admin
        print(f"   -> Sẽ tạo {9} người dùng ADMIN...")
        for i in range(9):
            admin_username = f"admin{admin_count}"
            new_user_id = next(gen)
            manager_salary_type = 'daily'
            manager_salary = random.randint(800000, 900000)
            
            # Tuple dữ liệu bây giờ có 20 giá trị, khớp với 20 cột của câu lệnh INSERT
            admin_record = (
                new_user_id, fake.name(), admin_username, hashed_password.decode('utf-8'),
                fake.email(), random.choice(['Nam', 'Nữ']), fake.date_of_birth(minimum_age=25, maximum_age=55),
                fake.address(), 
                1, # office_id (giả định)
                1, # department_id (giả định)
                admin_role_id,
                fake.date_between(start_date='-10y'),
                manager_salary_type, 
                manager_salary, # salary_per_day
                'active', 
                datetime.now(), # created_at
                user_id,        # created_by
                datetime.now(), # updated_at
                user_id,        # updated_by
                fake.phone_number()
            )
            cursor.execute(sql_insert_user, admin_record)
            print(f"   ✓ Đã tạo Admin: {admin_username}")
            admin_count += 1
        
        connect_mysql.commit()
        print(f"\n✅ Hoàn thành! Đã sinh {cursor.rowcount} dữ liệu người dùng ADMIN.")

    except mysql.connector.Error as error:
        print(f"❌ Lỗi khi thực thi: {error}")
    finally:
        if connect_mysql and connect_mysql.is_connected():
            cursor.close()
            connect_mysql.close()
            print("📦 Đã đóng kết nối database.")
            

generate_admin_users()

B1
🚀 Bắt đầu sinh dữ liệu cho người dùng ADMIN...
buoc3
   -> Sẽ tạo 9 người dùng ADMIN...
❌ Lỗi khi thực thi: 1264 (22003): Out of range value for column 'salary_per_day' at row 1
📦 Đã đóng kết nối database.


In [15]:

  connect_mysql.close()
  print("📦 Đã đóng kết nối database.")

📦 Đã đóng kết nối database.
